In [0]:
%pip install -U -qqqq mlflow databricks-openai databricks-agents
dbutils.library.restartPython()

In [0]:
dbutils.widgets.text("catalog_name", "btafur_catalog")
dbutils.widgets.text("schema_name", "default")
dbutils.widgets.text("model_name", "quickstart_agent")


catalog_name = dbutils.widgets.get("catalog_name")
schema_name = dbutils.widgets.get("schema_name")
model_name = dbutils.widgets.get("model_name")
dbutils.widgets.text("experiment_name", f"/Users/{dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()}/{model_name}_{catalog_name}")
experiment_name = dbutils.widgets.get("experiment_name")

In [0]:
import mlflow

registered_model_name = f"{catalog_name}.{schema_name}.{model_name}"

mlflow.set_experiment(experiment_name)

In [0]:
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint
from pkg_resources import get_distribution
from quickstart_agent import LLM_ENDPOINT_NAME
import datetime

mlflow.set_registry_uri("databricks-uc")
resources = [
    DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME),
]

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

with mlflow.start_run(run_name=f"sentiment_agent_{timestamp}"):
    logged_model_info = mlflow.pyfunc.log_model(
        artifact_path="agent",
        python_model="quickstart_agent.py",
        extra_pip_requirements=[
            f"databricks-connect=={get_distribution('databricks-connect').version}"
        ],
        resources=resources
    )

In [0]:
dbutils.jobs.taskValues.set(key = "logged_run_id", value = logged_model_info.run_id)